In [1]:
import pandas as pd
import os, inspect
from sklearn.utils import shuffle
import numpy as np
from statsmodels.stats.weightstats import DescrStatsW

import warnings
warnings.filterwarnings("ignore")

os.chdir('/Users/xiaodanxu/Documents/SynthFirm.nosync/CFS')

In [2]:
CFS_df = pd.read_csv('cfs-2017-puf-csv.csv', sep = ',')
print(CFS_df.columns)
print(len(CFS_df))
CFS_df.head(5)

Index(['SHIPMT_ID', 'ORIG_STATE', 'ORIG_MA', 'ORIG_CFS_AREA', 'DEST_STATE',
       'DEST_MA', 'DEST_CFS_AREA', 'NAICS', 'QUARTER', 'SCTG', 'MODE',
       'SHIPMT_VALUE', 'SHIPMT_WGHT', 'SHIPMT_DIST_GC', 'SHIPMT_DIST_ROUTED',
       'TEMP_CNTL_YN', 'EXPORT_YN', 'EXPORT_CNTRY', 'HAZMAT', 'WGT_FACTOR'],
      dtype='object')
5978523


,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,TEMP_CNTL_YN,EXPORT_YN,EXPORT_CNTRY,HAZMAT,WGT_FACTOR
0,1,6,99999,06-99999,6,260,06-260,326,4,43,5,4380,391,54,60,N,N,N,N,328.3
1,2,49,482,49-482,47,314,47-314,4541,3,43,14,56,4,1524,1810,N,N,N,N,8425.3
2,3,6,348,06-348,6,348,06-348,4231,4,34,5,255,440,2,5,N,N,N,N,9120.7
3,4,6,260,06-260,6,99999,06-99999,212,4,11,5,250,44912,30,35,N,N,N,N,20.9
4,5,45,273,45-273,45,273,45-273,45431,4,19,5,46,73,9,11,N,N,N,H,1733.8


In [ ]:
# load additional data to check
io_2017 = read_csv('data_2017io_filtered.csv')
naics_lookup = read_csv('corresp_naics6_n6io_sctg_revised.csv')

In [5]:
# clean the data

df_clean = CFS_df.loc[CFS_df.EXPORT_YN == 'N'] # Cleans out international exports

# def removeOutliers(sName, df):
#     # Computing IQR
#     Q1 = df[sName].quantile(0.25)
#     Q3 = df[sName].quantile(0.75)
#     IQR = Q3 - Q1

#     # Filtering Values between Q1-1.5IQR and Q3+1.5IQR
#     filtered = df.query(sName + '<= (@Q3 + 3 * @IQR)')
    
#     return filtered

# df_clean = removeOutliers('SHIPMT_WGHT', df_clean)
# df_clean = removeOutliers('SHIPMT_DIST_GC', df_clean)
# print(len(df_clean))

In [6]:
# estimate value density
lb_to_ton = 0.0005

df_clean.loc[:, 'SHIPMT_WGHT_scaled'] = df_clean.loc[:, 'SHIPMT_WGHT'] * \
df_clean.loc[:, 'WGT_FACTOR'] * lb_to_ton # in tons
df_clean.loc[:, 'SHIPMT_VALUE_scaled'] = df_clean.loc[:, 'SHIPMT_VALUE'] * df_clean.loc[:, 'WGT_FACTOR']
df_clean.loc[:, 'unitcost'] = df_clean.loc[:, 'SHIPMT_VALUE'] / \
df_clean.loc[:, 'SHIPMT_WGHT'] / lb_to_ton # in $/ton

print(df_clean.loc[:, 'SHIPMT_WGHT_scaled'].sum() * 2000)
print(df_clean.loc[:, 'SHIPMT_VALUE_scaled'].sum() / 1000000)
value_density_by_sctg = df_clean.groupby(['SCTG'])[['SHIPMT_WGHT_scaled', 'SHIPMT_VALUE_scaled']].sum()
value_density_by_sctg = value_density_by_sctg.reset_index()
value_density_by_sctg.loc[:, 'unitcost'] = value_density_by_sctg.loc[:, 'SHIPMT_VALUE_scaled'] / \
value_density_by_sctg.loc[:, 'SHIPMT_WGHT_scaled'] # in $/ton

value_density_by_sctg = value_density_by_sctg[['SCTG', 'unitcost']]
value_density_by_sctg.columns = ['Commodity_SCTG', 'UnitCost']
value_density_by_sctg.to_csv('data_unitcost_cfs2017.csv', index = False)

value_density_by_sctg.head(5)

23772729574972.4
13443011.544721209


,Commodity_SCTG,UnitCost
0,00,654.388298
1,01,2303.976289
2,01-05,244.701008
3,02,151.905213
4,03,787.351637


In [9]:
def weighted_std(df):
    stats = DescrStatsW(df['unitcost'], weights=df['SHIPMT_WGHT_scaled'], ddof=1)
    w_mean = stats.mean
    w_std = stats.std
    return(pd.Series([w_mean, w_std]))
value_density_by_sctg_2 = df_clean.groupby(['SCTG']).apply(weighted_std) 
value_density_by_sctg_2 = value_density_by_sctg_2.reset_index()
value_density_by_sctg_2.head(10)

,SCTG,0,1
0,00,654.388298,10960.923868
1,01,2303.976289,7669.093378
2,01-05,244.701008,938.520429
3,02,151.905213,316.533528
4,03,787.351637,8916.220077
5,04,404.383378,1519.481372
6,05,3874.318295,4812.683942
7,06,1552.698054,3482.067426
8,06-09,503.060907,1737.033675
9,07,1149.043617,6159.501996


In [10]:
print(df_clean['SCTG'].unique())
value_density_by_sctg_zone = df_clean.groupby(['SCTG', 'ORIG_CFS_AREA'])[['SHIPMT_WGHT_scaled', 'SHIPMT_VALUE_scaled']].sum()

value_density_by_sctg_zone = value_density_by_sctg_zone.reset_index()
value_density_by_sctg_zone.loc[:, 'unitcost'] = value_density_by_sctg_zone.loc[:, 'SHIPMT_VALUE_scaled'] / \
value_density_by_sctg_zone.loc[:, 'SHIPMT_WGHT_scaled']
value_density_by_sctg_zone.loc[value_density_by_sctg_zone['SCTG'] == '15-19', 'SCTG'] = '16'
# value_density_by_sctg_zone.loc[value_density_by_sctg_zone['SCTG'] == '15-19', 'SCTG'] = '16'
to_drop = ['35-38', '25-30', '31-34', '10-14', '06-09', 
           '20-24', '01-05', '39-43']
value_density_by_sctg_zone = \
value_density_by_sctg_zone[~value_density_by_sctg_zone['SCTG'].isin(to_drop)]
value_density_by_sctg_zone = value_density_by_sctg_zone[['SCTG', 'ORIG_CFS_AREA', 'SHIPMT_WGHT_scaled', 'unitcost']]
value_density_by_sctg_zone.columns = ['Commodity_SCTG', 'ORIG_CFS_AREA', 'Capacity', 'UnitCost']

value_density_by_sctg_zone.head(5)
value_density_by_sctg_zone.to_csv('data_unitcost_by_zone_cfs2017.csv', index = False)

['43' '34' '11' '19' '40' '36' '33' '38' '28' '35' '37' '26' '24' '32'
 '18' '20' '21' '07' '39' '04' '27' '08' '41' '30' '06' '23' '31' '29'
 '25' '05' '12' '13' '03' '10' '17' '22' '09' '02' '14' '01' '15-19' '15'
 '35-38' '25-30' '31-34' '10-14' '06-09' '20-24' '01-05' '00' '39-43']


In [12]:
def weighted_quantile(df, quantile=0.5):
    df_sorted = df.sort_values('SHIPMT_WGHT')
    cumsum = df_sorted['SHIPMT_WGHT_scaled'].cumsum()
    cutoff = df_sorted['SHIPMT_WGHT_scaled'].sum() * quantile
    value = df_sorted[cumsum >= cutoff]['SHIPMT_WGHT'].iloc[0]
    return(value)
load_by_sctg = df_clean.groupby(['SCTG']).apply(weighted_quantile)
load_by_sctg = load_by_sctg.reset_index()
load_by_sctg.loc[load_by_sctg['SCTG'] == '15-19', 'SCTG'] = '16'
load_by_sctg = load_by_sctg[~load_by_sctg['SCTG'].isin(to_drop)]
# load_by_sctg = df_clean.groupby(['SCTG'])[['SHIPMT_WGHT']].quantile(0.9)

load_by_sctg.columns = ['Commodity_SCTG', 'SHIPMT_WGHT']
# value_density_by_sctg.loc[:, 'unitcost'] = value_density_by_sctg.loc[:, 'SHIPMT_VALUE_scaled'] / \
# value_density_by_sctg.loc[:, 'SHIPMT_WGHT_scaled']

# value_density_by_sctg = value_density_by_sctg[['SCTG', 'unitcost']]
# value_density_by_sctg.columns = ['Commodity_SCTG', 'UnitCost']
load_by_sctg.to_csv('max_load_per_shipment_50percent.csv', index = False)
load_by_sctg.head(5)

,Commodity_SCTG,SHIPMT_WGHT
0,00,191836
1,01,42667
3,02,61436
4,03,53481
5,04,49567


In [8]:


count_by_sctg = df_clean.groupby(['SCTG'])[['SHIPMT_WGHT']].count()
count_by_sctg.columns = ['sample_size']
count_by_sctg = count_by_sctg.reset_index()
# value_density_by_sctg.loc[:, 'unitcost'] = value_density_by_sctg.loc[:, 'SHIPMT_VALUE_scaled'] / \
# value_density_by_sctg.loc[:, 'SHIPMT_WGHT_scaled']
print(load_by_sctg)
# value_density_by_sctg = value_density_by_sctg[['SCTG', 'unitcost']]
# value_density_by_sctg.columns = ['Commodity_SCTG', 'UnitCost']
# load_by_sctg.to_csv('max_load_per_shipment.csv', index = False)

     SCTG   SHIPMT_WGHT
0      00  3.293821e+05
1      01  5.767900e+04
2   01-05  4.735505e+06
3      02  1.056784e+06
4      03  7.935642e+04
5      04  6.686740e+04
6      05  4.457212e+04
7      06  5.465700e+04
8   06-09  7.388256e+05
9      07  5.552600e+04
10     08  4.534650e+04
11     09  2.553990e+04
12     10  5.659847e+04
13  10-14  4.497826e+06
14     11  2.428828e+05
15     12  8.490560e+04
16     13  1.887796e+05
17     14  2.019734e+07
18     15  3.415059e+07
19  15-19  1.964647e+07
20     17  2.386482e+05
21     18  7.170562e+04
22     19  6.186245e+04
23     20  1.894808e+05
24  20-24  1.643619e+06
25     21  1.162560e+04
26     22  2.061531e+05
27     23  4.666600e+04
28     24  4.544125e+04
29     25  1.557918e+05
30  25-30  2.297028e+05
31     26  7.265260e+04
32     27  1.232228e+05
33     28  3.960145e+04
34     29  2.370002e+04
35     30  2.246280e+04
36     31  6.963800e+04
37  31-34  2.972222e+06
38     32  5.282250e+04
39     33  4.339200e+04
40     34  2.544

In [10]:
value_fraction_by_origin = df_clean.groupby(['SCTG', 'ORIG_CFS_AREA'])[['SHIPMT_VALUE_scaled']].sum()
value_fraction_by_origin = value_fraction_by_origin.reset_index()

# value_fraction_by_origin.head(20)

value_fraction_by_dest = df_clean.groupby(['SCTG', 'DEST_CFS_AREA'])[['SHIPMT_VALUE_scaled']].sum()
value_fraction_by_dest = value_fraction_by_dest.reset_index()

value_fraction_by_dest.head(20)

,SCTG,DEST_CFS_AREA,SHIPMT_VALUE_scaled,value_fraction
0,00,01-99999,64704.1,0.000019
1,00,02-99999,47026734.5,0.014062
2,00,04-38060,986343.9,0.000295
3,00,05-99999,92954525.5,0.027796
4,00,06-260,762399.2,0.000228
5,00,06-348,3936573.5,0.001177
6,00,06-41740,42453719.0,0.012695
7,00,06-488,4380614.4,0.001310
8,00,06-99999,386260.2,0.000116
9,00,08-216,7658459.7,0.002290


In [16]:
cfs_to_faf_lookup = pd.read_csv('CFS_FAF_LOOKUP.csv', sep = ',')
cfs_to_faf_lookup.head(5)
value_fraction_by_origin_faf = pd.merge(value_fraction_by_origin, 
                                        cfs_to_faf_lookup, 
                                        left_on = 'ORIG_CFS_AREA',
                                        right_on = 'ST_MA', how = 'left')

value_fraction_by_dest_faf = pd.merge(value_fraction_by_dest, 
                                        cfs_to_faf_lookup, 
                                        left_on = 'DEST_CFS_AREA',
                                        right_on = 'ST_MA', how = 'left')

value_fraction_by_origin_faf = value_fraction_by_origin_faf.dropna(subset = ['FAF'])
value_fraction_by_origin_faf.loc[:, 'value_fraction'] = value_fraction_by_origin_faf.loc[:, 'SHIPMT_VALUE_scaled'] / \
value_fraction_by_origin_faf.groupby('SCTG')['SHIPMT_VALUE_scaled'].transform('sum')


value_fraction_by_dest_faf = value_fraction_by_dest_faf.dropna(subset = ['FAF'])
value_fraction_by_dest_faf.loc[:, 'value_fraction'] = value_fraction_by_dest_faf.loc[:, 'SHIPMT_VALUE_scaled'] / \
value_fraction_by_dest_faf.groupby('SCTG')['SHIPMT_VALUE_scaled'].transform('sum')

output_attr = ['SCTG', 'FAF', 'value_fraction']
output_attr_label = ['Commodity_SCTG', 'FAF', 'value_fraction']
value_fraction_by_origin_faf = value_fraction_by_origin_faf[output_attr]
value_fraction_by_origin_faf.columns = output_attr_label

value_fraction_by_dest_faf = value_fraction_by_dest_faf[output_attr]
value_fraction_by_dest_faf.columns = output_attr_label

value_fraction_by_origin_faf.to_csv('producer_value_fraction_by_faf.csv', index = False)
value_fraction_by_dest_faf.to_csv('consumer_value_fraction_by_faf.csv', index = False)